In [5]:
import os
os.environ["CEA_USE_SITE_PACKAGES"] = '1'
from CEA_Wrap import Fuel, Oxidizer, RocketProblem

from utils import bar_to_psi

Propellants

In [6]:
ipa = Fuel("C3H8O,2propanol", temp=298)  # Liquid IPA at room temperature
n2o = Oxidizer("N2O", temp=298)        # Nitrous oxide at room temperature
# TODO: not sure if there's a way to pull the Stoichiometric out of these two, 
# would be neat to plug in to graph by mixture instead of phi

In [7]:
CHAMBER_PRESSURE_BAR = 20   # Chamber pressure in bar
EXIT_PRESSURE_BAR = 1.4     # Sea level + some margin
PHI = 1.9                   # (Actual fuel/oxidiser) / (Stoichiometric fuel/oxidiser)

Pressure ratio

In [8]:
pressure_ratio = CHAMBER_PRESSURE_BAR / EXIT_PRESSURE_BAR
pressure_ratio

14.285714285714286

Rocket Problem

In [9]:
# See page 25 of manual for parameter description (.venv/lib/python3.13/site-packages/CEA_Wrap/assets/CEA_Users_Manual_and_Program_Description.pdf) 
problem = RocketProblem(
    pressure=bar_to_psi(CHAMBER_PRESSURE_BAR),      # Pressure taken in PSI
    materials=[ipa, n2o],                           # Propellants
    phi=PHI,                                        # (actual F/O) / (stoichiometric F/O)
    pip=pressure_ratio,                             # Pressure ratio (Pc/Pe)
)

Run the analysis

In [10]:
results = problem.run()

In [11]:
results.__dict__.keys()

dict_keys(['prod_c', 'massf', 'p', 't_p', 'c_p', 't', 't_t', 'c_t', 'h', 't_h', 'c_h', 'rho', 't_rho', 'c_rho', 'son', 't_son', 'c_son', 'visc', 't_visc', 'c_visc', 'cond', 't_cond', 'c_cond', 'pran', 't_pran', 'c_pran', 'mw', 't_mw', 'c_mw', 'm', 't_m', 'c_m', 'condensed', 't_condensed', 'c_condensed', 'cp', 't_cp', 'c_cp', 'gammas', 't_gammas', 'c_gammas', 'gamma', 't_gamma', 'c_gamma', 'isp', 't_isp', 'ivac', 't_ivac', 'cf', 't_cf', 'cstar', 'mach', 'o_f', 'phi', 'ae', 't_ae', 'pip', 't_pip', 'prod_t', 'prod_e', 'dLV_dLP_t', 't_dLV_dLP_t', 'c_dLV_dLP_t', 'dLV_dLT_p', 't_dLV_dLT_p', 'c_dLV_dLT_p'])

Key results

In [15]:
print(f"Chamber Temperature (K):            {results.c_t:.1f}")
print(f"Chamber Pressure (bar):             {results.c_p:.1f}")
print(f"Exit Temperature (K):               {results.t:.1f}")
print(f"Exit Pressure (bar):                {results.p:.1f}")
print(f"Specific Impulse (s):               {results.isp:.1f}")
print(f"Characteristic Velocity (m/s):      {results.cstar:.1f}")
print(f"Thrust Coefficient:                 {results.cf:.3f}")
print(f"Specific Heat Ratio (gamma):        {results.gamma:.2f}")
print(f"Area Ratio (Expansion ratio):       {results.ae:.4f}")
print(f"Final Mach Number:                  {results.mach:.4f}")


Chamber Temperature (K):            2805.5
Chamber Pressure (bar):             20.0
Exit Temperature (K):               1673.1
Exit Pressure (bar):                1.4
Specific Impulse (s):               210.8
Characteristic Velocity (m/s):      1550.2
Thrust Coefficient:                 1.334
Specific Heat Ratio (gamma):        1.26
Area Ratio (Expansion ratio):       2.7091
Final Mach Number:                  2.3601


In [11]:
for var, value in results.__dict__.items():
    print(f"{var}: {value}")

prod_c: {'CO': 0.21436, 'CO2': 0.03989, 'H': 0.00712, 'H2': 0.14533, 'H2O': 0.18868, 'NH3': 1e-05, 'NO': 0.00037, 'N2': 0.40126, 'O': 8e-05, 'OH': 0.00285, 'O2': 4e-05}
massf: False
p: 1.0
t_p: 11.165
c_p: 20.0
t: 1559.7
t_t: 2531.0
c_t: 2805.5
h: -1912.0
t_h: -143.51
c_h: 423.66
rho: 0.17646
t_rho: 1.2113
c_rho: 1.9517
son: 846.51
t_son: 1065.1
c_son: 1115.3
visc: 6.6653e-05
t_visc: 9.1259e-05
c_visc: 9.7787e-05
cond: 0.24542000000000003
t_cond: 0.48023999999999994
c_cond: 0.6384299999999999
pran: 0.47192
t_pran: 0.38628
c_pran: 0.35
mw: 22.883
t_mw: 22.832
c_mw: 22.762
m: 22.883
t_m: 22.832
c_m: 22.762
condensed: False
t_condensed: False
c_condensed: False
cp: 1.7377
t_cp: 2.0327
c_cp: 2.285
gammas: 1.2645
t_gammas: 1.2307
c_gammas: 1.2138
gamma: 1.264512645
t_gamma: 1.232090691
c_gamma: 1.21707726
isp: 220.31600407747197
t_isp: 108.57288481141691
ivac: 247.04383282364932
t_ivac: 196.77879714576963
cf: 1.3942
t_cf: 0.68703
cstar: 1550.2
mach: 2.5532
o_f: 3.4692
phi: 1.9
ae: 3.3828
t_

Exhaust composition (top 5 species)

In [10]:
print("\n=== Major Exhaust Products (mol fraction) ===")
products = [(species, fraction) for species, fraction in results.prod_e.items()]
products.sort(key=lambda x: x[1], reverse=True)

for species, fraction in products[:5]:
    print(f"{species:10}: {fraction:.4f}")


=== Major Exhaust Products (mol fraction) ===
N2        : 0.4036
CO        : 0.1904
H2        : 0.1728
H2O       : 0.1680
CO2       : 0.0652
